Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [0]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [0]:
import os
data_root = './data'
pickle_file = os.path.join(data_root, 'notMNIST.pickle')
print("{}".format(pickle_file))

In [0]:
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [0]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale
def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

In [0]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [0]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [0]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [0]:
# Randomly picks a batch of training and validation data
def get_data(self, dataset, labels):
  n = len(dataset)
  perm = np.random.randint(n, size=self.batch_size)
  return dataset[perm], labels[perm]

def get_hidden_dim(self):
  n_dim = image_size//(self.pool_st*self.pool_st)*image_size//(self.pool_st*self.pool_st)*self.n_depth
  return n_dim

In [0]:
# CONVOLUTIONAL NEURAL MODEL:
# Softmax Score: Input => Conv1 => ReLU1 => MaxPool1 => Conv2 => ReLU2 => MaxPool2 => NN(FC-ReLU3) => FC
def define_state_conv_nn(self):
  with self.graph.as_default():
    # Input Layer placeholders for data and labels
    self.dataset_ph = tf.placeholder(tf.float32,
                                     shape=(None, image_size, image_size, num_channels), name="dataset")
    self.labels_ph = tf.placeholder(tf.float32,
                                    shape=(None, num_labels), name="labels")
    self.l2_reg_ph = tf.placeholder(tf.float32, name= "l2_reg")
    self.dropout_ph = tf.placeholder(tf.float32, name= "dropout")
    
    # Use truncated normal initializer: stddev = 0.1
    # Bound the weights to prevent exploding gradient problem
    tr_norm_init = tf.truncated_normal_initializer(stddev=0.1)
    # Conv1:
    with tf.variable_scope("Conv1", initializer=tr_norm_init) as scope:
      conv1_weights = tf.get_variable("Weights", [self.patch_size, self.patch_size, num_channels, self.n_depth])
      conv1_biases    = tf.get_variable("Biases", [self.n_depth], initializer=tf.zeros_initializer)
    # Conv2:
    with tf.variable_scope("Conv2", initializer=tr_norm_init) as scope:
      conv2_weights = tf.get_variable("Weights", [self.patch_size, self.patch_size, self.n_depth, self.n_depth])
      conv2_biases    = tf.get_variable("Biases", [self.n_depth], initializer=tf.zeros_initializer)

    n_dim = self.get_hidden_dim()
    # NN: Fully Connected Layer => Non Linear xForm (ReLU)
    with tf.variable_scope("NN", initializer=tr_norm_init) as scope:
      nn_weights = tf.get_variable("Weights", [n_dim, self.hidden_size])
      nn_biases    = tf.get_variable("Biases", [self.hidden_size], initializer=tf.zeros_initializer)
    # Softmax Score Layer: Fully Connected Projection Layer
    with tf.variable_scope("Softmax", initializer=tr_norm_init) as scope:
      sm_weights = tf.get_variable("Weights", [self.hidden_size, num_labels])
      sm_biases    = tf.get_variable("Biases", [num_labels], initializer=tf.zeros_initializer)

    return

In [0]:
# Softmax Score: Input => Conv1 => ReLU1 => MaxPool1 => Conv2 => ReLU2 => MaxPool2 => NN(FC-ReLU3) => FC
def define_computation_conv_nn(self):
  with self.graph.as_default():
    # Conv1
    with tf.variable_scope("Conv1", reuse=True) as scope:
      conv1_weights = tf.get_variable("Weights")
      conv1_biases = tf.get_variable("Biases")
      
    conv1 = tf.nn.conv2d(self.dataset_ph, conv1_weights, [1, 1, 1, 1], padding='SAME') 
    relu1 = tf.nn.relu(conv1 + conv1_biases, "ReLU1")
    # Prevent Overfitting: Dropout at Activation and L2-Reg of Weights
    conv1d = tf.nn.dropout(relu1, self.dropout_ph)
    conv1_loss = tf.nn.l2_loss(conv1_weights)
    # MaxPool1
    maxpool1 = tf.nn.max_pool(conv1d, ksize=[1, self.kernel_size, self.kernel_size, 1],
                              strides=[1, self.pool_st, self.pool_st, 1], padding='SAME')
    
    # Conv2
    with tf.variable_scope("Conv2", reuse=True) as scope:
      conv2_weights = tf.get_variable("Weights")
      conv2_biases = tf.get_variable("Biases")
      
    conv2 = tf.nn.conv2d(maxpool1, conv2_weights, [1, 1, 1, 1], padding='SAME') 
    relu2 = tf.nn.relu(conv2 + conv2_biases, "ReLU2")
    # Prevent Overfitting: Dropout at Activation and L2-Reg of Weights
    conv2d = tf.nn.dropout(relu2, self.dropout_ph)
    conv2_loss = tf.nn.l2_loss(conv2_weights)
    # MaxPool2
    maxpool2 = tf.nn.max_pool(conv2d, ksize=[1, self.kernel_size, self.kernel_size, 1],
                              strides=[1, self.pool_st, self.pool_st, 1], padding='SAME')

    # NN
    with tf.variable_scope("NN", reuse=True) as scope:
      nn_weights = tf.get_variable("Weights")
      nn_biases = tf.get_variable("Biases")
    # Reshape maxpool2 to feed into the fully connected Neural Network Layer
    n_dim = np.prod(maxpool2.get_shape().as_list()[1:])
    nn_input = tf.reshape(maxpool2, [-1, n_dim])
    Z_nn = tf.matmul(nn_input, nn_weights) + nn_biases
    A_nn = tf.nn.relu(Z_nn, "ReLU3")
    # Prevent Overfitting: Dropout at Activation and L2-Reg of Weights
    Ad_nn = tf.nn.dropout(A_nn, self.dropout_ph)
    nn_loss = tf.nn.l2_loss(nn_weights)

    # SoftmaxScore
    with tf.variable_scope("Softmax", reuse=True) as scope:
      sm_weights = tf.get_variable("Weights")
      sm_biases    = tf.get_variable("Biases")
    Z_sm = tf.matmul(Ad_nn, sm_weights) + sm_biases
    # Prevent Overfitting: Dropout at Activation and L2-Reg of Weights
    logits = tf.nn.dropout(Z_sm, self.dropout_ph)  
    sm_loss = tf.nn.l2_loss(sm_weights)
    
    # SoftmaxScore to Prediction
    self.prediction_op = tf.nn.softmax(logits, name="Prediction")
    # Cross Entropy Loss
    XEnt = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=self.labels_ph)
  
    # Scalar Loss
    self.loss_op = tf.reduce_mean(XEnt) + self.l2_reg_ph*(conv1_loss + conv2_loss + nn_loss + sm_loss)
  
    # Accuracy
    P = tf.argmax(self.prediction_op, axis=1)
    L = tf.argmax(self.labels_ph, axis=1)
    self.accuracy_op = 100.0 * tf.reduce_mean(tf.to_float(tf.equal(P,L)))

    # Optimizer with exponential decay of learning rate
    n_steps = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(self.lr, n_steps, self.decay_steps, self.decay_rate)
    self.train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(self.loss_op, global_step=n_steps)

  return

In [0]:
def run_epochs_conv_nn(self):
  best_val_epoch = 0
  best_val_acc      = -float('inf')

  with tf.device('/cpu:0'), tf.Session(graph=self.graph) as s:
    initer = tf.global_variables_initializer()
    s.run(initer)
    print("Initialized")
    for epoch in range(self.n_epochs):
      max_steps = len(train_dataset)/self.batch_size
      loss = []
      acc  = []
      for step in range(max_steps):
        # Generate a minibatch.
        batch_data, batch_labels = self.get_data(train_dataset, train_labels)
        # Dictionary: Feeds Training Minibatch.
        feed_dict = {
          self.dataset_ph:    batch_data,
          self.labels_ph:       batch_labels,
          self.l2_reg_ph:       self.l2_reg,
          self.dropout_ph:   self.dropout
        }
        _, tr_loss, tr_acc = s.run( [self.train_op, self.loss_op, self.accuracy_op], feed_dict=feed_dict)
        loss.append(tr_loss)
        acc.append(tr_acc)
        if (step % self.n_validate == 0):
          print("Epoch=%d: Step=%d: Training Loss=%.2f: Accuracy=%.2f%%"
                %(epoch, step, np.mean(loss), np.mean(acc)))

      # Compute validation accuracy realized after a full epoch run
      feed_dict = {
        self.dataset_ph: valid_dataset,
        self.labels_ph: valid_labels,
        self.l2_reg_ph:0.0,
        self.dropout_ph:1.0
      }
      val_acc = s.run(self.accuracy_op, feed_dict=feed_dict)
      print("----------------")
      print("Epoch=%d: Training Loss=%.2f: Accuracy=%.2f%%: Validation Accuracy: %.2f%%\n"
            %(epoch, np.mean(loss), np.mean(acc), val_acc))
          
      # Remember the epoch when best validation accuracy was realized
      # Stop early if average validation accuracy is not improving for a few steps
      if val_acc > best_val_acc:
        best_val_epoch = step
        best_val_acc = val_acc
          
      if epoch >= (best_val_epoch + self.early_stop):
        print("Step %d: Best_Epoch %d: Early_Stop %d" % (step, best_val_epoch, self.early_stop))
        print("Terminate training early. Validation accuracy has flattened at %.2f%%"% best_val_acc)
        break
      
    print("=================")
    feed_dict = {
      self.dataset_ph: test_dataset,
      self.labels_ph: test_labels,
      self.l2_reg_ph:0.0,
      self.dropout_ph:1.0
    }
    test_acc = s.run( self.accuracy_op, feed_dict=feed_dict)
    print("Training Completed: Test Accuracy: %.2f%%\n" % test_acc)
    print("=================")

In [0]:
# Fixed Dimensions
# image_size = 28
# num_labels = 10
# num_channels = 1
class ImageModel():
  l2_reg            = 0.01
  dropout        = 0.9
  lr                     = 0.1
  decay_rate   = 0.9
  decay_steps = 1000
  kernel_size   = 2
  pool_st         = 2
  patch_size   = 5
  n_depth       = 16
  hidden_size = 64
  batch_size   = 128
  n_validate   = 32
  n_epochs     = 10
  early_stop   = 2
  get_hidden_dim = get_hidden_dim
  define_state = define_state_conv_nn
  define_computation = define_computation_conv_nn
  get_data = get_data
  run_epochs = run_epochs_conv_nn

  def __init__(self):
    self.graph = tf.Graph()
    
imMod = ImageModel()    

In [0]:
imMod.define_state()
imMod.define_computation()
imMod.run_epochs()

Validation Accuracy: 89.20%
Test Accuracy: 94.94%
Based on the latest hyperparameters:
l2_reg            = 0.01
dropout        = 0.9
lr                     = 0.1
decay_rate   = 0.9
decay_steps = 1000
kernel_size   = 2
pool_st         = 2
patch_size   = 5
n_depth       = 16
hidden_size = 64
batch_size   = 128
n_epochs     = 10